In [6]:
# 实现图算法
# 使用两个类，Graph类存储包含所有带点的主列表，Vertex类存储每个顶点的信息

# 实现Vertex类
class Vertex():
    def __init__(self, key):
        # 顶点的id
        self.id = key
        # 顶点的连接,以字典的形式存储，key是顶点对象，value是权重
        self.connectedTo = {}
        
        # 针对广度优先搜索的辅助变量
        self.color = 'white'  # 顶点的颜色，白色表示未访问，灰色表示访问过但未探索，黑色表示访问过且探索过
        self.dist = 0  # 顶点的距离
        self.pred = None  # 顶点的前驱
    
    def addNeighbor(self, nbr, weight=0):
        # 添加邻接顶点
        self.connectedTo[nbr] = weight
        
    def __str__(self):
        # 返回顶点的连接信息
        return str(self.id) + ' connectedTo: ' + str([x.id for x in self.connectedTo])
    
    def getConnections(self):
        # 返回所有连接的顶点
        return self.connectedTo.keys()
    
    def getId(self):
        # 返回顶点的id
        return self.id
    
    def getWeight(self, nbr):
        # 返回顶点的权重
        return self.connectedTo[nbr]
    
    # 针对广度优先搜索的辅助函数
    # 针对广度优先搜索的辅助函数
    # 针对广度优先搜索的辅助函数
    def setColor(self, color):
        self.color = color
        
    def getColor(self):
        return self.color
    
    def setDistance(self, d):
        self.dist = d
        
    def getDistance(self):
        return self.dist
    
    def setPred(self, p):
        self.pred = p
        
    def getPred(self):
        return self.pred


# 实现Graph类
class Graph():
    def __init__(self):
        # 图的顶点列表
        self.vertList = {}
        # 顶点的个数
        self.numVertices = 0
    
    def addVertex(self, key):
        # 添加顶点
        self.numVertices += 1
        newVertex = Vertex(key)
        self.vertList[key] = newVertex
        return newVertex
    
    def getVertex(self, n):
        # 返回顶点
        if n in self.vertList:
            return self.vertList[n]
        else:
            return None
        
    def __contains__(self, n):
        # 判断顶点是否在图中
        return n in self.vertList
    
    def addEdge(self, f, t, cost=0):
        # 添加边
        # f是起点，t是终点，cost是权重
        if f not in self.vertList:
            nv = self.addVertex(f)
        if t not in self.vertList:
            nv = self.addVertex(t)
        self.vertList[f].addNeighbor(self.vertList[t], cost)
        
    def getVertices(self):
        # 返回所有顶点
        return self.vertList.keys()
    
    def __iter__(self):
        # 迭代器
        # 返回所有顶点
        # 形式为每一个顶点相连的所以顶点
        return iter(self.vertList.values())

In [7]:
# 测试图算法
g = Graph()
for i in range(6):
    g.addVertex(i)
print(g.vertList)

{0: <__main__.Vertex object at 0x0000020E03BACC70>, 1: <__main__.Vertex object at 0x0000020E03882E50>, 2: <__main__.Vertex object at 0x0000020E03C924C0>, 3: <__main__.Vertex object at 0x0000020E03F73AC0>, 4: <__main__.Vertex object at 0x0000020E03C34130>, 5: <__main__.Vertex object at 0x0000020E03EB6F40>}


In [8]:
# 队列
class Queve:
    def __init__(self):
        self.items = []

    def isEmpty(self):
        return self.items == []

    def enqueue(self, item):  # 入队
        self.items.insert(0, item)

    def dequeue(self):  # 出队
        return self.items.pop()

    def size(self):
        return len(self.items)


In [9]:
# 这里解决一个词梯问题
# 构建一个图，每个单词是一个顶点，如果两个单词只有一个字母不同，则两个单词之间有一条边
def buildGraph(wordList):
    d = {}  # 创建一个字典，key是单词，value是单词对应的顶点
    g = Graph()  # 创建一个图
    # 遍历单词列表,创建词桶
    for word in wordList:
        g.addVertex(word)
        for i in range(len(word)):
            bucket = word[:i] + '_' + word[i+1:]
            bucket = d.get(bucket, [])
            bucket.append(word)
            d[bucket] = bucket
    # 遍历词桶，为词桶中的单词添加边
    for bucket in d.values():
        for word1 in bucket:
            for word2 in bucket:
                if word1 != word2:
                    g.addEdge(word1, word2)
    return g 

In [10]:
# 实现广度优先搜索
def bfs(gragh, start):
    start.setDistance(0)  # 起始点的距离为0
    start.setPred(None)  # 起始点的前驱为None
    vertQueue = Queve()  # 创建一个队列
    vertQueue.enqueue(start)  # 起始点入队
    while vertQueue.size() > 0:
        currentVert = vertQueue.dequeue()  # 出队
        for nbr in currentVert.getConnections():
            # 遍历当前顶点的所有邻接顶点
            nbr.setColor('gray')  # 邻接顶点变为灰色,表示已经访问过
            nbr.setDistance(currentVert.getDistance() + 1)  # 邻接顶点的距离为当前顶点的距离加1
            nbr.setPred(currentVert)  # 邻接顶点的前驱为当前顶点
            vertQueue.enqueue(nbr)  # 邻接顶点入队
        currentVert.setColor('black')  # 当前顶点变为黑色，表示已经探索过

# 回溯，找到从起始点到终点的路径
def traverse(y):
    x = y
    while x.getPred():
        print(x.getId())
        x = x.getPred()
    print(x.getId())
    

In [11]:
# 优化广义优先搜索算法，不必遍历所有的顶点，只需要遍历到终点即可
# 返回路径
def bfsAdjust(gragh, start, end):
    # start是起始点，end是终点
    start.setDistance(0)  # 起始点的距离为0
    start.setPred(None)  # 起始点的前驱为None
    vertQueue = Queve()  # 创建一个队列
    # 起始点入队
    vertQueue.enqueue(start)
    while vertQueue.size() > 0:
        currentVert = vertQueue.dequeue()  # 出队
        if currentVert == end:
            # 如果当前顶点是终点，返回路径
            traverse(currentVert)
            return
        for nbr in currentVert.getConnections():
            # 遍历当前顶点的所有邻接顶点
            if nbr.getColor() == 'white':
                # 如果邻接顶点未访问过
                nbr.setColor('gray')  # 邻接顶点变为灰色,表示已经访问过
                nbr.setDistance(currentVert.getDistance() + 1)
                nbr.setPred(currentVert)
                vertQueue.enqueue(nbr)
        currentVert.setColor('black')
    return None

In [14]:
# 解决8数码问题
def bfs8(start, end):
    # start是起始状态，end是终止状态
    # start和end是一个长度为9的元组，表示一个状态
    gragh = Graph()
    current = Vertex(start)
    gragh.addVertex(current)
    vertQueue = Queve()
    vertQueue.enqueue(current)
    while vertQueue.size() > 0:
        current = vertQueue.dequeue()
        if current.getId() == end:
            return current.getDistance()
        for i in range(9):
            if current.getId()[i] == 0:
                break
        list_current = list(current.getId())  # 将当前状态转换为列表
        if i - 3 >= 0:
            # 上移
            self